# Imports

In [7]:
# pdf scanner
import PyPDF2
from pypdf import PdfReader
import pdfminer
# arrays
import pandas as pd
import numpy as np
# os
import os
# NLP
    # regex
import re
    # unicode
import unidecode

## ✅ Pulling Images (only required once)

In [8]:
# reader object
# reader = PdfReader('data.pdf')

In [9]:
# number of pages within data pdf
# num_pages = len(reader.pages)

In [10]:
# images pulled from pdf
# reader = PdfReader("data.pdf")
# for page in reader.pages:
#     for image in page.images:
#         with open(image.name, "wb") as fp:
#             fp.write(image.data)

# Retrieving data from all pdfs

## Not using isolated pdf version of retrieval

### Proceeding with compiled PDF, code remains for documentation purposes

In [11]:
# using pdfs seperated for \n inside the text

# importing required modules
# bodies = []
# names = ['dali','warhol','banksy','marcchagall','picasso','rembrandt','kaws','leoneardtsuguharufoujita','yayoi-kusama','zhangdaqian']
# # creating a pdf file object
# for name in names:
#     open_name = './' + name + '.pdf'
#     pdfFileObj = open(open_name, 'rb')
#     # creating a pdf reader object
#     pdfReader = PyPDF2.PdfReader(pdfFileObj)
#     # printing number of pages in pdf file
#     num_pages = len(pdfReader.pages)
#     print(f"Number of pages: {num_pages}")
#     # loop through all pages and extract text
#     for page_num in range(num_pages):
#         pageObj = pdfReader.pages[page_num]
#         text = pageObj.extract_text()
#         bodies.append(text)
#         print(f"Page {page_num+1}:\n{name}")
#         # validate the process body of text is correct format
#         if type(text) is str:
#             print('Successfully pulled text from page.\n')
#         else:
#             print('Failed to pull text.')
#     # closing the pdf file object
#     pdfFileObj.close()
#     print(f'+++++++++++++++++++++\nFinished text from {name} pdf\n+++++++++++++++++++++\n')


In [12]:
# using as a whole pdf because it removes the line breaks \n

# importing required modules
bodies = []

# creating a pdf file object
pdfFileObj = open('./data.pdf', 'rb')

# creating a pdf reader object
pdfReader = PyPDF2.PdfReader(pdfFileObj)

# printing number of pages in pdf file
num_pages = len(pdfReader.pages)
print(f"Number of pages: {num_pages}")

    # loop through all pages and extract text
for page_num in range(num_pages):
    pageObj = pdfReader.pages[page_num]
    text = pageObj.extract_text()
    bodies.append(text)
    print(f"Page {page_num+1}:\ndata.pdf")
    # validate the process body of text is correct format
    if type(text) is str:
        print('Successfully pulled text from page.\n')
    else:
        print('Failed to pull text.')
# closing the pdf file object
pdfFileObj.close()


Number of pages: 219
Page 1:
data.pdf
Successfully pulled text from page.

Page 2:
data.pdf
Successfully pulled text from page.

Page 3:
data.pdf
Successfully pulled text from page.

Page 4:
data.pdf
Successfully pulled text from page.

Page 5:
data.pdf
Successfully pulled text from page.

Page 6:
data.pdf
Successfully pulled text from page.

Page 7:
data.pdf
Successfully pulled text from page.

Page 8:
data.pdf
Successfully pulled text from page.

Page 9:
data.pdf
Successfully pulled text from page.

Page 10:
data.pdf
Successfully pulled text from page.

Page 11:
data.pdf
Successfully pulled text from page.

Page 12:
data.pdf
Successfully pulled text from page.

Page 13:
data.pdf
Successfully pulled text from page.

Page 14:
data.pdf
Successfully pulled text from page.

Page 15:
data.pdf
Successfully pulled text from page.

Page 16:
data.pdf
Successfully pulled text from page.

Page 17:
data.pdf
Successfully pulled text from page.

Page 18:
data.pdf
Successfully pulled text from page.

In [13]:
# function for isolating and pulling jp & ch characters
def remove_ch_jp_characters(text):
  """
  Removes all chinese characters from a string
  """
  pattern = re.compile(r'[\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff\uf900-\ufaff\uff66-\uff9f]')
  return pattern.sub('', text)

In [14]:
# decoding letters with accents into regular letters & removing character symbols
body_clean = []
for body in bodies:
    body = unidecode.unidecode(body)
    # removing the title of pdf from the body of text
    body = body.replace('Price Database31 October 2023','')
    body_clean.append(body)

In [15]:
# separating each entry by artist name
names = ['Zhang Daqian', 'Andy Warhol', 'Banksy', 'Salvador Dali','Marc Chagell','Pablo Picasso',
         'Rembrandt van Rijn','KAWS','Leonard Tsuguharu Foujita','Yayoi Kusama']
# container for all entries
new_body = []
# container for each artist that has multiple names in the entry
banksy_clean = []
marc_clean = []
pablo_clean = []
rembrandt_clean = []
dali_clean = []
for body in body_clean:
    for name in names:

        # isolating the entries that contain multiple counts of name in entry
            # the text in the if statement is the combined text where the name is repeated
            # these varie between title of piece or within the lot description

        # Banksy / KAWS
        if name == 'Banksy' or 'KAWS':
            if 'Only Banksy' in body:
                banksy_clean.append(body)
            elif 'Banksy and 21st Century Editions' in body:
                banksy_clean.append(body)

        # Marc Chagell
        if name == 'Marc Chagell':
            if 'Marc Chagall, Colour of Life' in body:
                marc_clean.append(body)
        # Dali
        if name == 'Salvador Dali':
            if 'After Salvador Dali' in body:
                dali_clean.append(body)
            elif 'SALVADOR DALI\' "Genesis"' in body:
                dali_clean.append(body)
            elif 'Currier & Ives as Interpreted by Salvador Dali' in body:
                dali_clean.append(body)
        # Picasso
        if name == 'Pablo Picasso':
            if 'After Pablo Picasso' in body:
                pablo_clean.append(body)
        # Rembrandt
        if name == 'Rembrandt van Rijn':
            if 'School of Rembrandt van Rijn' in body:
                rembrandt_clean.append(body)
            elif 'Circle of Rembrandt van Rijn' in body:
                rembrandt_clean.append(body)
            elif 'Studio of Rembrandt van Rijn' in body:
                rembrandt_clean.append(body)
            elif 'Follower of Rembrandt van Rijn' in body:
                rembrandt_clean.append(body)

        # All other values that do not have repeated names
        else:
            entries = body.split(f"{name}")[1:]
        for entry in entries:
            entry = entry.strip()
            # adding an additional space back in between the name and piece_name
            a = ''.join([name+' ',entry])
            a = a.replace('\x00','-')
            new_body.append(a)
            print(a)
            print("="*30)


Banksy Sale Ends (V.2)screenprint in colors on wove paperHeight 56.01 x Width 76 cm.Height 22.05 x Width 29.92 in.2017Edition: Edition 238/50015 December 202211 months agoartnet AuctionsContemporary Finds - [Lot 138555]est. 20,000 u 30,000 USD30,000USD
Banksy Donuts (Strawberry)screenprint in coloursHeight 55.6 x Width 75.8 cm.Height 21.89 x Width 29.843 in.2009Edition: 281/29913 December 202211 months agoBonhams LondonPrints and Multiples - [Lot 00086]est. 20,000 u 30,000 GBP29,595GBPest. 24,749 u 37,124 USD36,622USD
Banksy Napalmscreenprint in coloursHeight 50 x Width 70 cm.Height 19.685 x Width 27.559 in.2005Edition: 58/15013 December 202211 months agoBonhams LondonPrints and Multiples - [Lot 00085]est. 20,000 u 30,000 GBP25,500GBPest. 24,749 u 37,124 USD31,555USD
Banksy NOLA (Green/Blue)screenprint in colorsHeight 76 x Width 56 cm.Height 29.921 x Width 22.047 in.2008Edition: AP aside from ed. 28913 December 202211 months agoForum AuctionsThe British Sale (with Dreweatts 1759) - [Lo

In [16]:
# Removing entries from new_body that do not have complete data
new_body = [body for body in new_body if len(body) >= 130]

## Isolated Lists of Entries that have the name of artist within the data.

### Banksy / KAWS

In [17]:
# removing duplicates
banksy_clean = list(set(banksy_clean))

In [18]:
banksy_ = []
for i in banksy_clean:
    # splitting by new line
    entries = i.split('\n')
    for entry in entries:
        # adding space after artist name for ease of regex
        entry = entry.replace('Banksy','Banksy ')
        entry = entry.replace('KAWS','KAWS ')
        # removing double spaces
        entry = entry.replace('  ', ' ')
        banksy_.append(entry)

# cleaned banksy entries with multiple appearances of names
banksy_

['Banksy Bomb Middle Englandscreenprint in coloursHeight 34.5 x Width 98.5 cm.Height 13.583 x Width 38.78 in.2003Edition: 50008 November 202212 months agoForum AuctionsOnly Banksy - [Lot 00013]est. 14,000 u 18,000 GBP14,000GBPest. 16,228 u 20,864 USD16,228USD',
 'Banksy Barcodeon wove paperHeight 49.7 x Width 70 cm.Height 19.567 x Width 27.559 in.2004Edition: 60008 November 202212 months agoForum AuctionsOnly Banksy - [Lot 00008]est. 25,000 u 35,000 GBP24,500GBPest. 28,978 u 40,570 USD28,399USD| 2% est',
 'Banksy Banksquiat (Black) (Signed)screenprint in coloursHeight 75 x Width 70 cm.Height 29.528 x Width 27.559 in.2019Edition: 30008 November 202212 months agoForum AuctionsOnly Banksy - [Lot 00004]est. 60,000 u 80,000 GBP58,000GBPest. 69,549 u 92,732 USD67,230USD| 3% est',
 'Banksy HMVscreenprint on paperHeight 29 x Width 44 cm.Height 11.417 x Width 17.323 in.2003Edition: 60029 October 2022about 1 year agoSBI Art Auction Co., Ltd.The 54th SBI Art Auction|Modern andContemporary Art - [

### Rembrandt

In [19]:
# removing duplicates
rembrandt_clean = list(set(rembrandt_clean))

In [20]:
rembrandt_ = []
for i in rembrandt_clean:
    # splitting by new line
    entries = i.split('\n')
    for entry in entries:
        # adding space after artist name for ease of regex
        entry = entry.replace('Rembrandt van Rijn','Rembrandt van Rijn ')
        # removing double spaces
        entry = entry.replace('  ', ' ')
        rembrandt_.append(entry)

# cleaned rembrandt entries with multiple appearances of names
rembrandt_

["Rembrandt van Rijn The Adoration of theShepherds: A Night Piece (B.,Holl. 46; New Holl. 300; H. 255)etchingHeight 15 x Width 19.6 cm.Height 5.906 x Width 7.717 in.165709 December 202211 months agoSotheby's LondonOld Master Prints (Online) - [Lot 00060]est. 25,000 u 35,000 GBP30,240GBPest. 30,784 u 43,098 USD37,236USD",
 "Studio of Rembrandt van Rijn A boy readingoil on canvasHeight 75.6 x Width 59.5 cm.Height 29.764 x Width 23.425 in.09 December 202211 months agoChristie's OnlineOld Master Paintings and Sculpture - [Lot00146]est. 10,000 u 15,000 GBP44,100GBPest. 12,313 u 18,470 USD54,303USD| 194% est",
 "Rembrandt van Rijn The Triumph of Mordecai (B.,Holl. 40; New Holl. 185; H. 172)etchingHeight 17.8 x Width 21.7 cm.Height 7.008 x Width 8.543 in.164109 December 202211 months agoSotheby's LondonOld Master Prints (Online) - [Lot 00063]est. 8,000 u 12,000 GBP16,380GBPest. 9,851 u 14,776 USD20,169USD| 36% est",
 'Rembrandt van Rijn The Hundred Guilder Printetching, engraving, and drypoin

### Marc Chagall

In [21]:
marc_ = []
for i in marc_clean:
    # splitting by new line
    entries = i.split('\n')
    for entry in entries:
        # adding space after artist name for ease of regex
        entry = entry.replace('Marc Chagall','Marc Chagall ')
        # removing double spaces
        entry = entry.replace('  ', ' ')
        marc_.append(entry)

# cleaned marc chagall entries with multiple appearances of names
marc_

["Marc Chagall Couple au clair de luneoil, tempera and coloured inks on canvasHeight 100 x Width 78 cm.Height 39.37 x Width 30.709 in.1980AE198101 December 202211 months agoChristie's Hong KongMarc Chagall , Colour of Life: Works Formerlyfrom the Artist's Estate (Part II) - [Lot 00108]est. 4,800,000 u 6,800,000 HKD6,174,000HKDest. 617,141 u 874,283 USD793,797USD",
 "Marc Chagall Idylle aux fleurs et fruits devantla fenetre a Parisgouache, tempera, india ink and pastel on paperHeight 44.4 x Width 30.5 cm.Height 17.48 x Width 12.008 in.197601 December 202211 months agoChristie's Hong KongMarc Chagall , Colour of Life: Works Formerlyfrom the Artist's Estate (Part II) - [Lot 00113]est. 550,000 u 850,000 HKD2,016,000HKDest. 70,714 u 109,285 USD259,199USD| 137% est",
 'Marc Chagall L\'inspiration du peintre pour"chere Vava"pastel, coloured crayons, india ink and gouacheover lithograph in black on paperHeight 44.7 x Width 67.2 cm.Height 17.598 x Width 26.457 in.195701 December 202211 months a

### Pablo Picasso

In [22]:
pablo_ = []
for i in pablo_clean:
    # splitting by new line
    entries = i.split('\n')
    for entry in entries:
        # adding space after artist name for ease of regex
        entry = entry.replace('Pablo Picasso','Pablo Picasso ')
        # removing double spaces
        entry = entry.replace('  ', ' ')
        pablo_.append(entry)

# cleaned pablo picasso entries with multiple appearances of names
pablo_

['After Pablo Picasso After Pablo Picasso WoolTapestrywool tapestryHeight 194.31 x Width 220.98 cm.Height 76.5 x Width 87 in.196508 December 202211 months agoNeal Auction CompanyMid-Century, Modern, Contemporary Art &Design Online Auction - [Lot 00253]est. 2,000 u 3,000 USD12,800USD| 327% est',
 'Pablo Picasso Quatre personnagespen and brush ink on paperHeight 50.5 x Width 65.5 cm.Height 19.882 x Width 25.787 in.196808 December 202211 months agoKarl & Faber KunstauktionenGmbHAuction 314: Modern Art | Evening Sale - [Lot00730]est. 300,000 u 350,000 EUR275,000EURest. 316,522 u 369,276 USD290,145USD| 8% est',
 'After Pablo Picasso BUSTE DE FEMME EN BLANC,1930color aquatint etchingHeight 55.5 x Width 45 cm.Height 21.85 x Width 17.717 in.1930Edition: 1/12508 December 202211 months agoHampel Fine Art AuctionsYoung Collectors: Modern & Contemporary Art- [Lot 00587]est. 9,500 u 12,000 EUR9,710EURest. 10,023 u 12,660 USD10,244USD',
 'Pablo Picasso Hibouink on pageHeight 31.4 x Width 23.7 cm.Hei

### Salvador Dali

In [23]:
dali_clean

["\nSalvador DaliLes Amours Jaunes (Michel etLopsinger 693u702; Field 74u15)the complete set of ten drypoint etchings withgildingHeight 39.8 x Width 30 x Depth 2.4 cm.Height 15.669 x Width 11.811 x Depth 0.945 in.197413 December 202211 months agoBonhams LondonPrints and Multiples - [Lot 00025]est. 5,000 u 7,000 GBP9,562GBPest. 6,187 u 8,662 USD11,832USD| 37% est\nAfter Salvador DaliTwo After Salvador Dali SilverGilt and Enamel MenagereMolluscsilverHeight 20.96 cm.Height 8.25 in.195707 December 202211 months agoDoyle New YorkDoyle+Design(r) - [Lot 00056]est. 20,000 u 30,000 USD25,200USD\nSalvador DaliCabinet anthropomorphiquebronzeHeight 60 cm.Height 23.622 in.1973Edition: 267/33005 December 202211 months agoSotheby's ParisArt Impressionniste et Moderne Day Auction(Online) - [Lot 00085]est. 15,000 u 20,000 EUR13,860EURest. 15,744 u 20,992 USD14,548USD| 8% est\nSalvador Dali(i) Don Quichotte (ii) Dedication(i) & (ii) felt-tip pen on a page of gerard marx'sbook, dali de dragerHeight 30.1 

In [24]:
dali_ = []
for i in dali_clean:
    # splitting by new line
    entries = i.split('\n')
    for entry in entries:
        # adding space after artist name for ease of regex
        entry = entry.replace('Dali','Dali ')
        # removing double spaces
        entry = entry.replace('  ', ' ')
        dali_.append(entry)

# cleaned dali entries with multiple appearances of names
dali_ = dali_[1:]
dali_

['Salvador Dali Les Amours Jaunes (Michel etLopsinger 693u702; Field 74u15)the complete set of ten drypoint etchings withgildingHeight 39.8 x Width 30 x Depth 2.4 cm.Height 15.669 x Width 11.811 x Depth 0.945 in.197413 December 202211 months agoBonhams LondonPrints and Multiples - [Lot 00025]est. 5,000 u 7,000 GBP9,562GBPest. 6,187 u 8,662 USD11,832USD| 37% est',
 'After Salvador Dali Two After Salvador Dali SilverGilt and Enamel MenagereMolluscsilverHeight 20.96 cm.Height 8.25 in.195707 December 202211 months agoDoyle New YorkDoyle+Design(r) - [Lot 00056]est. 20,000 u 30,000 USD25,200USD',
 "Salvador Dali Cabinet anthropomorphiquebronzeHeight 60 cm.Height 23.622 in.1973Edition: 267/33005 December 202211 months agoSotheby's ParisArt Impressionniste et Moderne Day Auction(Online) - [Lot 00085]est. 15,000 u 20,000 EUR13,860EURest. 15,744 u 20,992 USD14,548USD| 8% est",
 "Salvador Dali (i) Don Quichotte (ii) Dedication(i) & (ii) felt-tip pen on a page of gerard marx'sbook, dali de dragerH

In [25]:
# combing all cleaned entries
add_ins = dali_ + rembrandt_ + marc_ + pablo_ + banksy_
len(add_ins)

128

## Connecting multiple name counts with the rest of values that did not need extra cleaning

In [26]:
print(len(new_body))
new_body.extend(add_ins)
print(len(new_body))

909
1037


In [27]:
# reaffirming all cleaned entries are removed 
new_body = [body for body in new_body if len(body) >= 130]

## Isolating values by regex needs

In [28]:
# pre regex
# 1. if name count < 1, drop value (DONE)

# regex
    # 2. Make depth optional
    # 3. make foreign currency (estimate & hammer price) optional
    # 4.

# vars
# new_body = collective of all values
# has_fc
    # fc_has_depth
    # fc_no_depth

In [29]:
# seperating by IF has foreign currency
foreign_currency = ['HKD','EUR','NZD','AUD','GBP','KRW','CNY','JPY','CHF','SGD','CAD','TWD','INR','THB','RUB','SEK','DKK','NOK',
                    'PLN','ZAR','MXN','BRL','TRY','AED','SAR','QAR','KWD','OMR','BHD','JOD','ILS','EGP','IDR','MYR','PHP','CZK',
                    'HUF','CLP','ARS','COP','PEN']
# seperating by IF has depth
# Depth as a keyword

In [38]:
has_foreign_currency = [body for body in new_body if any(currency in body for currency in foreign_currency)]
len(has_foreign_currency)

794

## Manual fixes for names that were not separated for one reason or another

In [39]:
has_foreign_currency

['Banksy Donuts (Strawberry)screenprint in coloursHeight 55.6 x Width 75.8 cm.Height 21.89 x Width 29.843 in.2009Edition: 281/29913 December 202211 months agoBonhams LondonPrints and Multiples - [Lot 00086]est. 20,000 u 30,000 GBP29,595GBPest. 24,749 u 37,124 USD36,622USD',
 'Banksy Napalmscreenprint in coloursHeight 50 x Width 70 cm.Height 19.685 x Width 27.559 in.2005Edition: 58/15013 December 202211 months agoBonhams LondonPrints and Multiples - [Lot 00085]est. 20,000 u 30,000 GBP25,500GBPest. 24,749 u 37,124 USD31,555USD',
 'Banksy NOLA (Green/Blue)screenprint in colorsHeight 76 x Width 56 cm.Height 29.921 x Width 22.047 in.2008Edition: AP aside from ed. 28913 December 202211 months agoForum AuctionsThe British Sale (with Dreweatts 1759) - [Lot00124]est. 120,000 u 150,000 GBP110,000GBPest. 148,496 u 185,620 USD136,121USD| 8% est',
 'Banksy Queen Vicscreenprint in colorsHeight 70.7 x Width 49.7 cm.Height 27.835 x Width 19.567 in.2003Edition: 50013 December 202211 months agoForum Auc

In [32]:
for i, body in enumerate(has_foreign_currency):
    if '\nSalvador Dali' in body:
        print(f'Place in List: {i}')
        body = body.replace('\nSalvador Dali','Salvador Dali')

In [33]:
has_foreign_currency

['Banksy Donuts (Strawberry)screenprint in coloursHeight 55.6 x Width 75.8 cm.Height 21.89 x Width 29.843 in.2009Edition: 281/29913 December 202211 months agoBonhams LondonPrints and Multiples - [Lot 00086]est. 20,000 u 30,000 GBP29,595GBPest. 24,749 u 37,124 USD36,622USD',
 'Banksy Napalmscreenprint in coloursHeight 50 x Width 70 cm.Height 19.685 x Width 27.559 in.2005Edition: 58/15013 December 202211 months agoBonhams LondonPrints and Multiples - [Lot 00085]est. 20,000 u 30,000 GBP25,500GBPest. 24,749 u 37,124 USD31,555USD',
 'Banksy NOLA (Green/Blue)screenprint in colorsHeight 76 x Width 56 cm.Height 29.921 x Width 22.047 in.2008Edition: AP aside from ed. 28913 December 202211 months agoForum AuctionsThe British Sale (with Dreweatts 1759) - [Lot00124]est. 120,000 u 150,000 GBP110,000GBPest. 148,496 u 185,620 USD136,121USD| 8% est',
 'Banksy Queen Vicscreenprint in colorsHeight 70.7 x Width 49.7 cm.Height 27.835 x Width 19.567 in.2003Edition: 50013 December 202211 months agoForum Auc

In [34]:
has_depth = [body for body in new_body if 'Depth' in body]

In [35]:
len(has_depth)

75

In [36]:
depth_has_fc = [body for body in has_depth if any(fc in body for fc in foreign_currency)]
depth_no_fc = [body for body in has_depth if not any(fc in body for fc in foreign_currency)]

In [37]:
print(len(depth_has_fc), len(depth_no_fc))

53 22
